
# Asset Utilization

This Jupyter notebook transform raw data of utilization into time series data:

## Purpose
Easier access to utilization data

## Outputs
- Utilization data will be available in the Tag service.


## Imports and Setup
This section imports all the required libraries and environment variables

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import re
import requests
import copy
import asyncio
import json
import random
import string
import requests
import scrapbook as sb
from copy import deepcopy
from datetime import datetime, timedelta, timezone
from collections import defaultdict
from typing import Any, Callable, Dict, List, Tuple
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
from systemlink.clients.niapm.api.assets_api import AssetsApi
from systemlink.clients.niapm.api.utilization_api import UtilizationApi
from systemlink.clients.niapm.api.policy_api import PolicyApi
from systemlink.clients.niapm.models.query_assets_request import QueryAssetsRequest
from systemlink.clients.niapm.models.query_asset_utilizations_request import QueryAssetUtilizationsRequest
from systemlink.clients.niapm.models.utilization_time_interval_model import UtilizationTimeIntervalModel

from systemlink.clients.nisysmgmt.api.systems_api import SystemsApi
from systemlink.clients.nisysmgmt.models.query_systems_request import QuerySystemsRequest

from systemlink.clients.nitag.api.tags_api import TagsApi
from systemlink.clients.nitag.models.tag import Tag
from systemlink.clients.nitag.models.tag_update import TagUpdate
from systemlink.clients.nitag.models.tag_value import TagValue
from systemlink.clients.nitag.models.tag_type import TagType
from systemlink.clients.nitag.models.tag_list_and_merge_flag import TagListAndMergeFlag
from systemlink.clients.nitag.models.timestamped_tag_value import TimestampedTagValue

from systemlink.httputilizationalgorithm import TimeIntervalGenerator
from systemlink.messagebus.workspace_translator import translate_ids_to_names

In [ ]:
api_key = os.getenv("SYSTEMLINK_API_KEY")
http_url = os.getenv('SYSTEMLINK_HTTP_URI')
os.environ['SKYLINE_USER_SERVICES_URL'] = http_url

# Session configuration

In [ ]:
session = requests.Session()
session.headers.update({
    "Content-Type": "application/json",
    'x-ni-api-key': api_key,
})
 
retry_strategy = Retry(
    total=3,  # total retry attempts
    status_forcelist=[500, 502, 503, 504],  # which errors to retry
    allowed_methods=["HEAD", "GET", "OPTIONS", "POST", "PUT", "DELETE"],  # methods to retry
    backoff_factor=2  # wait 1s, 2s, 4s ... between retries
)

# Mount the retry strategy to http and https
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)
 

# Input parameters

### 1. `target_date`
- **Expected Value:** Date/time for the day to calculate utilization, or empty string to use the default.
- **Format:** ISO-8601 string (e.g., '2025-11-12T23:59:59').
- **Impact:** Determines which day’s utilization is computed. If "", utilization is calculated for yesterday

### 2. `target_workspaces`
- **Expected Value:** List of workspace IDs to generate utilization for.
- **Format:** Array of workspace ID values (e.g., ["ws_123", "ws_456"]).
- **Impact:** Limits utilization generation to the specified workspaces. If empty ([]), utilization is generated for all workspaces.

### 3. `tag_chunk_size`
- **Expected Value:** Number of tag updates to include in a single request/query batch.
- **Format:** Positive integer.
- **Impact:** Increasing the values reduces the number of requests, keep in mind that the tag service can process up to ~1000 tag updates per second.

### 4. `delay_between_tag_batches`
- **Expected Value:** Delay to wait between consecutive tag update requests/batches.
- **Format:** Positive float.
- **Impact:** Throttles request rate to keep tag update throughput within the Tag service limit ~1000 tag updates per second.

### 5. `debug_tag_prefix`
- **Expected Value:** String prefix used for debug
- **Format:** String.
- **Impact:** After debug tags can be identified and deleted separately from production tags.

### 6. `policy_start_time`
- **Expected Value:** Time of day (UTC) that defines when the workday starts for the utilization/work-time policy.
- **Format:** Time-only string in 24-hour UTC format, HH:MM (e.g., "05:00").
- **Impact:** The utilization percentage calculation based on working hours policy.

### 7. `policy_end_time`
- **Expected Value:** Time of day (UTC) that defines when the workday ends for the utilization/work-time policy.
- **Format:** Time-only string in 24-hour UTC format, HH:MM (e.g., "05:00").
- **Impact:** The utilization percentage calculation based on working hours policy.

In [ ]:
target_date = ""
# target_date = "2025-11-12T23:59:59"
target_workspaces = ["a99d186f-8e2b-499c-9707-a30e2174c007"]
# target_workspaces = []


tag_chunk_size = 900
delay_between_tag_batches = 0.9
debug_tag_prefix = ""
# debug_tag_prefix = '_' + ''.join(random.choices(string.ascii_letters + string.digits, k=10))

policy_start_time="05:00"
policy_end_time="04:59"

# Configuration

In [ ]:
class JobConfig:
    MAX_TAG_UPDATES_PER_SECOND = 1000

    def __init__(
        self,
        target_date="",
        target_workspaces=[],
        tag_chunk_size=500,
        delay_between_tag_batches=1,
        debug_tag_prefix="",
        policy_start_time="05:00",
        policy_end_time="04:59",
    ):
        
        self.tag_chunk_size = tag_chunk_size
        self.delay_between_tag_batches = delay_between_tag_batches
        self.policy_start_time = policy_start_time
        self.policy_end_time = policy_end_time

        self.start_date, self.end_date = self._compute_date_range(target_date)
        self.tag_middle_segment = f"Utilization{debug_tag_prefix}"

        self._validate_tag_service_capacity()

        self.target_workspaces = self.check_target_workspaces(target_workspaces)

        self.utilization_intervals, self.utilization_for_date = self.get_utilization_day()
        
        self.interval_group_function = lambda interval: interval.start.astype(datetime).strftime("%Y-%m-%d")

        self.property_not_available_string = 'Not Available'
        self.group_by_value = 'asset_identifier'
        
    def _compute_date_range(self, date_str: str):
        if date_str == "":
            yesterday = datetime.utcnow() - timedelta(days=1)
            start_date = yesterday.strftime("%Y-%m-%dT00:00:00")
            end_date = yesterday.strftime("%Y-%m-%dT23:59:59")
            return start_date, end_date

        start_date = date_str[:-1]
        end_date = date_str[:-9] + "23:59:59"
        return start_date, end_date

    def _validate_tag_service_capacity(self):
        """
                Validate tag update rate is within service capacity
                """
        if self.delay_between_tag_batches <= 0:
            raise ValueError(
                f"delay_between_tag_batches must be > 0, got {self.delay_between_tag_batches}."
            )

        rate = self.tag_chunk_size / self.delay_between_tag_batches
        if rate > self.MAX_TAG_UPDATES_PER_SECOND:
            raise ValueError(
                f"Max allowed is {self.MAX_TAG_UPDATES_PER_SECOND} tags/sec. "
            )
    
    def check_target_workspaces(self, target_workspaces):
        """
                All provided workspaces should be authorized+enabled
                """
        auth_api_key_endpoint = f"{http_url}/niauth/v1/auth"
        response = session.get(url=auth_api_key_endpoint)
        response.raise_for_status()
        data = response.json()
    
        workspaces = data["workspaces"]

        authorized_and_enabled_workspaces = [ws for ws in workspaces if ws.get("enabled") is True]
        if len(target_workspaces) == 0:
            return authorized_and_enabled_workspaces

        # Validate that every target id is authorized+enabled
        missing = []
        for target_id in target_workspaces:
            found = False
            for ws in authorized_and_enabled_workspaces:
                if ws.get("id") == target_id:
                    found = True
                    break
            if not found:
                missing.append(target_id)
    
        if len(missing) > 0:
            raise ValueError(
                "target_workspaces contains workspace ids that are not authorized and enabled: "
                + ", ".join(missing)
            )

        # Filter, preserving original order from authorized_and_enabled_workspaces
        filtered = []
        for ws in authorized_and_enabled_workspaces:
            ws_id = ws.get("id")
            for target_id in target_workspaces:
                if ws_id == target_id:
                    filtered.append(ws)
                    break
    
        return filtered

    def get_target_workspace_ids(self):
        return [ws['id'] for ws in self.target_workspaces]

    def get_utilization_day(self):
        group_by = 'DAY'
        time_interval_generator = TimeIntervalGenerator(self.start_date,
                                                        self.end_date,
                                                        group_by,
                                                        None,
                                                        policy_start_time,
                                                        policy_end_time)
        intervals = time_interval_generator.get_intervals()
        
        dates = [str(interval.start).split('T')[0] for interval in intervals]
        
        return intervals, dates[0]

In [ ]:
cfg = JobConfig(
    target_date=target_date,
    target_workspaces=target_workspaces,
    tag_chunk_size=tag_chunk_size,
    delay_between_tag_batches=delay_between_tag_batches,
    debug_tag_prefix=debug_tag_prefix,
    policy_start_time="05:00",
    policy_end_time="04:59",
)

In [ ]:
def adjust_datetime(date_time_str, whp_start_time):
    date_time = datetime.strptime(date_time_str, "%Y-%m-%d")
    year = date_time.year
    month = date_time.month
    day = date_time.day
    start_time = datetime.strptime(whp_start_time, '%H:%M').time()
    adjusted_timezone = timezone(timedelta(hours=0))
    target_datetime = datetime(year, month, day, start_time.hour, start_time.minute, start_time.second, tzinfo=adjusted_timezone)
    iso_utc_datetime = target_datetime.astimezone(timezone.utc).isoformat().replace('+00:00', 'Z')
    return iso_utc_datetime

## Implementation of compute_utilization function (do not touch)

In [ ]:
#Just delete this part when SDK will be ready

from systemlink.httputilizationalgorithm import date_time_helper, dataframe_utilization_algorithm
from systemlink.clients.niapm import ApiClient
import pandas as pd

async def compute_utilization(  # pylint: disable=too-many-arguments
        utilization_filter: str,
        asset_filter: str,
        intervals: List[date_time_helper.Interval],
        grouping_criteria: str,
        null_group_value: Any,
        startTime: str = None,
        endTime: str = None,
        hash_interval: Callable[[date_time_helper.Interval], Any] = lambda interval: interval.start.day) -> Tuple[Dict[Tuple[Any, Any], float], List[str]]:  # pylint: disable=line-too-long
    """
    Gets utilization data from the Asset Management service
    and computed the utilization percentages for the passed in criteria.
 
    :param utilization_filter: The filter to be used when querying the utilization history
    :type utilization_filter: str
    :param asset_filter: The filter that determines what assets should be taken
        into consideration when computing the utilization.
    :type asset_filter: str
    :param intervals: The intervals used to compute the utilization.
    :type intervals: List[date_time_helper.Interval]
    :param grouping_criteria: The name of a column from the input dataframe.
    :type grouping_criteria: str
    :param null_group_value: The value used to fill null like values for the
        column used for grouping.
    :type null_group_value: Any
    :param hash_interval: Function that determines the time granularity.
        Takes in a date_time_helper.Interval
        E.g: If the results should be grouped by months, this function
        should return the same value for any interval in the same month.
    :type hash_interval: Callable[[date_time_helper.Interval], Any])
    :return: A tuple with the computed utilization and the list of unused asset identifiers.
    :rtype: Tuple[Dict[Tuple[Any, Any], float], List[str]]
    """
    heartbeat_interval_minutes = 10
 
    raw_utilization = []
    async for chunk in query_raw_utilization(utilization_filter, asset_filter, startTime, endTime):
        raw_utilization.extend(chunk)
 
    asset_ids = await _query_asset_ids(asset_filter)
    if not raw_utilization:
        return {}, asset_ids
 
    if pd.__version__ >= "2.0.0":
        utilization_df = pd.DataFrame([u.to_dict() for u in raw_utilization], dtype=(str))  # pylint: disable=line-too-long
        for x in ['start_timestamp', 'end_timestamp', 'heartbeat_timestamp']:
            utilization_df[x] = pd.to_datetime(utilization_df[x], utc=True, format="ISO8601").dt.tz_localize(None)
    else:
        utilization_df = pd.DataFrame(
            [u.to_dict() for u in raw_utilization], dtype=("datetime64[ns]")
        )  # pylint: disable=line-too-long
 
    utilization_df[grouping_criteria].fillna(null_group_value, inplace=True)
 
    computed_utilization, used_asset_ids = dataframe_utilization_algorithm.compute(
        utilization_df,
        intervals,
        grouping_criteria,
        hash_interval,
        heartbeat_interval_minutes)
 
    unused_asset_ids = list(set(asset_ids) - set(used_asset_ids))
 
    return computed_utilization, unused_asset_ids
 
 
async def _query_asset_ids(asset_filter: str) -> List[str]:
    async with ApiClient() as api_client:
        assets_api = AssetsApi(api_client=api_client)
        query_assets_request = QueryAssetsRequest(filter=asset_filter, take=-1)
        query_assets_response = await assets_api.query_assets(query_assets=query_assets_request)
        return [asset.id for asset in query_assets_response.assets]
 
 
async def query_raw_utilization(
    utilization_filter: str,
    asset_filter: str,
    startTime: str,
    endTime: str,
    take: int = 10000
) -> List[object]:
    """
    Queries raw utilization data.
 
    :param utilization_filter: The filter to be used when querying the utilization history
    :type utilization_filter: str
    :param asset_filter: The filter that determines what assets should be taken
        into consideration when computing the utilization.
    :type asset_filter: str
    :param startTime: The start time for the query in ISO 8601 format.
    :type startTime: str
    :param endTime: The end time for the query in ISO 8601 format.
    :type endTime: str
    :param take: The maximum number of entries to get in one request. Maximum is 1000.
    :type take: int = 1000
    :return: A generator yielding chunks of utilization entries.
    :rtype: List[systemlink.assetmgmtclient.messages.UtilizationHistoryModel]
    """
    returned_assets_count = take
    async with ApiClient() as api_client:
        utilization_api = UtilizationApi(api_client=api_client)
 
        request = {
            "utilizationFilter": utilization_filter,
            "assetFilter": asset_filter,
            "continuationToken": "",
            "orderBy": "START_TIMESTAMP",
            "startTime": startTime,
            "endTime": endTime,
            "take": take,
        }
 
        while returned_assets_count == take:
            query_utilization_response = await utilization_api.query_asset_utilizations(
                query_body=request
            )  # pylint: disable=line-too-long
            utilizations = query_utilization_response.asset_utilizations
            if not utilizations:
                return
            yield utilizations
            returned_assets_count = len(utilizations)
            request['continuationToken'] = query_utilization_response.continuation_token

In [ ]:
def backfill_unused_assets_utilization(grouped_utilization, unused_asset_identifiers, date):
    filled = dict(grouped_utilization)
    all_asset_ids = set(asset_id for asset_id, _ in grouped_utilization.keys())
    all_asset_ids.update(unused_asset_identifiers)
    for asset_id in all_asset_ids:
        key = (asset_id, date)
        if key not in filled:
            filled[key] = {
                'percentage': 0.0,
                'grouping_value': asset_id,
                'interval_hash': date
            }

    return filled

## Ensure tags exists

In [ ]:
async def ensure_tags_exists(tags_metadata=[]):
    update_tags_endpoint = f"{http_url}/nitag/v2/update-tags"
    tag_template = {
        "metadata": {
            "properties": {
                "nitagRetention": "PERMANENT",
                "units": "%"
            }
        }
    }
    tags = []
    for metadata in tags_metadata:
        merged_properties = {
            **tag_template["metadata"]["properties"],
            **metadata.get("properties", {})
        }
        new_tag = {
            "path": metadata["path"],
            "type": "DOUBLE",
            "collectAggregates": True,
            "properties": merged_properties,
            "workspace": metadata["properties"]["workspace"],
        }
        tags.append(new_tag)
    chunked_tags = [tags[i:i + tag_chunk_size] for i in range(0, len(tags), tag_chunk_size)]
    payload_template = {"tags": [], "merge": True}
    for chunk in chunked_tags:
        payload = deepcopy(payload_template)
        payload["tags"].extend(chunk)
        response = session.post(url=update_tags_endpoint, json=payload)
        response.raise_for_status()

## Functions to generate filter string

In [ ]:
def create_asset_filter(assets, batch_size=500):
    return [
        "(" + " OR ".join(f'AssetIdentifier = "{asset['id']}"' for asset in assets[i:i+batch_size]) + ")"
        for i in range(0, len(assets), batch_size)
    ]

def create_minion_filter(systems, workspaces=[]):
    minion_ids = [f'Location.MinionId = "{minion["id"]}"' for minion in systems]
    filter_str = f'({" OR ".join(minion_ids)}) AND IsSystemController = True'
    if len(workspaces) == 0:
        return filter_str
    else:
        workspace_ids = [f'workspace = \"{workspace}\"' for workspace in workspaces]
        return f"{filter_str} AND ({" OR ".join(workspace_ids)})"

## Fetch paginated data

In [ ]:
async def fetch_all_assets(query_request, skip=0, take=-1):
    assets_api = AssetsApi()
    query_request.skip = skip
    query_request.take = take
 
    response = await assets_api.query_assets(query_assets=query_request)
    
    await assets_api.api_client.close()
    return response.to_dict()['assets']
    # return getattr(response, 'assets')
 
async def fetch_all_systems(query_request, skip=0, take=1000):
    system_api = SystemsApi()
    items = []
    query_request.skip = skip
    query_request.take = take
 
    response = await system_api.get_systems_by_query(query=query_request)
    items.extend(getattr(response, 'data'))
 
 
    while response.count == take:
        skip += take
        query_request.skip = skip
        response = await system_api.get_systems_by_query(query=query_request)
        items.extend(getattr(response, 'data'))
 
    await system_api.api_client.close()
    return items

## Aggregate different categories utilization

In [ ]:
def aggregateCategoryUtilization(input_data):
    # Dictionary to accumulate objects by assetIdentifier
    grouped_data = defaultdict(list)
    
    # Separate objects with and without "category" field
    non_category_items = [item for item in input_data if "category" not in item]
    category_items = [item for item in input_data if "category" in item]
    
    # Group category items by assetIdentifier
    for item in category_items:
        grouped_data[item['assetIdentifier']].append(item)
    
    # Consolidate duplicate entries
    consolidated_items = []
    for asset_id, items in grouped_data.items():
        if len(items) > 1:
            # Calculate mean percentage for duplicates
            mean_percentage = sum(item['percentage'] for item in items) / len(items)
            # Create a new consolidated object
            consolidated_item = items[0].copy()
            consolidated_item['percentage'] = mean_percentage
            del consolidated_item['category']
            consolidated_items.append(consolidated_item)
        else:
            # No duplicates, keep the single item as is
            consolidated_items.append(items[0])
    
    # Combine non-category items and consolidated category items
    result = non_category_items + consolidated_items
    return result

In [ ]:
async def patch_virtual_system_utilization(tags_metadata, date_str):
    tags_api = TagsApi()
    await ensure_tags_exists(tags_metadata=tags_metadata)
    
    timestamp_adjusted_by_timezone = adjust_datetime(
        date_time_str=date_str,
        whp_start_time=policy_start_time
    )
    tag_value = TagValue(value=str(0), type=TagType.DOUBLE)
    timestamped_tag_value = TimestampedTagValue(timestamp=timestamp_adjusted_by_timezone, value=tag_value)

    controlled_systems_tag_updates = []
    for metadata in tags_metadata:
        tag_update = TagUpdate(
            path= metadata['path'], 
            updates=[timestamped_tag_value], 
            workspace=metadata['properties']['workspace']
        )
        controlled_systems_tag_updates.append(tag_update)
    chunked_tag_updates = [controlled_systems_tag_updates[i:i + tag_chunk_size] for i in range(0, len(controlled_systems_tag_updates), tag_chunk_size)]
    for tag_updates in chunked_tag_updates:
        await tags_api.update_tag_current_values(updates=tag_updates)
        await asyncio.sleep(cfg.delay_between_tag_batches)
    await tags_api.api_client.close()

## Is valid tag

In [ ]:
def is_valid_tag_name(name):
    pattern = r'^[A-Za-z0-9\-\.\s\/\\()_\+\[\]{}\?~&\*,"\'`=‑\:]+$'
    is_valid = bool(re.match(pattern, name))
    return is_valid

In [ ]:
def get_utilization_settings(utilization: str):
    u = (utilization or "").strip().lower()

    if u == "configuration":
        return 'Category == "Configuration" or Category == "configuration"', "Configuration"
    if u == "test":
        return 'Category == "Test" or Category == "test"', "Test"

    return "", "All"

In [ ]:
def get_vendor_model_serial_number_settings(asset: Dict):
    vendor = asset['vendor_name']
    if vendor is None or vendor == "":
        vendor = str(asset['vendor_number'])
    model = asset['model_name']
    if model is None or model == "":
        model = str(asset['model_number'])
    serial_number = str(asset['serial_number'])

    return vendor, model, serial_number

In [ ]:
def group_systems_by_workspace(systems):
    grouped = defaultdict(list)
    for system in systems:
        workspace_id = system.get("workspace")
        grouped[workspace_id].append(system)
    return dict(grouped)

## Build workspace_system_map

In [ ]:
def split_and_enrich_for_workspace(ws_systems, ws_controllers):
    system_id_to_alias = {}
    for system in ws_systems:
        system_id = system.get("id", system.get("system_id"))
        if system_id is not None:
            system_id_to_alias[system_id] = system.get("alias")

    controller_minion_ids = set()
    for c in ws_controllers:
        minion_id = c["location"].get("minion_id")
        if minion_id is not None:
            controller_minion_ids.add(minion_id)

    controllers_with_alias = []
    for c in ws_controllers:
        c2 = deepcopy(c)
        minion_id = c2["location"].get("minion_id")
        if minion_id in system_id_to_alias:
            c2["alias"] = system_id_to_alias[minion_id]
        controllers_with_alias.append(c2)

    virtual_systems = []
    for system in ws_systems:
        system_id = system.get("id", system.get("system_id"))
        if system_id not in controller_minion_ids:
            virtual_systems.append(deepcopy(system))

    return controllers_with_alias, virtual_systems

def build_workspace_system_map(systems, system_controllers):
    systems_by_ws = defaultdict(list)
    controllers_by_ws = defaultdict(list)

    for system in systems:
        ws = system.get("workspace")
        if ws is not None:
            systems_by_ws[ws].append(system)

    for controller in system_controllers:
        ws = controller.get("workspace")
        if ws is not None:
            controllers_by_ws[ws].append(controller)

    _workspace_system_map = {}

    all_workspaces = set(systems_by_ws.keys()) | set(controllers_by_ws.keys())
    for ws in all_workspaces:
        controllers_with_alias, virtual_systems = split_and_enrich_for_workspace(
            systems_by_ws.get(ws, []),
            controllers_by_ws.get(ws, []),
        )
        _workspace_system_map[ws] = {
            "controllers_with_alias": controllers_with_alias,
            "virtual_systems": virtual_systems,
        }

    return _workspace_system_map

## Asset Utilization

In [ ]:
async def update_asset_utilization_tags(utilization=None, assets=[]):
    utilization_filter, category_prefix_for_tag = get_utilization_settings(utilization)
    
    tags_api = TagsApi()
    if assets:
        asset_id_to_asset_map = {}
        for asset in assets:
            asset_id_to_asset_map[asset['id']] = asset
    
        # ensure that tags are created for all assets
        asset_tags = []
        for asset in assets:
            location = asset['location']['minion_id'] if asset['location']['minion_id'] else asset['location']['physical_location']
            properties = {'name': str(asset['name']),
                          'id': str(asset['id']),
                          # 'minion_id': str(asset['location']['minion_id']),
                          'asset_type': str(asset['asset_type']),
                          'workspace': str(asset['workspace']),
                          'displayName': f"{asset['name']} - Utilization Daily Peak"}
    
            vendor, model, serial_number = get_vendor_model_serial_number_settings(asset)
    
            if not is_valid_tag_name(model) or not is_valid_tag_name(asset['serial_number']) or not is_valid_tag_name(vendor):
                print (f"Invalid asset {asset['id']} {asset['serial_number']} {model} {vendor}")
                continue       
            tag_path = f"Assets.{vendor}.{model}.{serial_number}.{cfg.tag_middle_segment}.Daily.All.{category_prefix_for_tag}"
            asset_tags.append({'path': tag_path, 'properties': properties})
        await ensure_tags_exists(asset_tags)
    
    
        asset_filters_list = create_asset_filter(assets)
        tag_updates = []
        for asset_filter in asset_filters_list:
            grouped_utilization, unused_asset_identifiers = await compute_utilization(
                utilization_filter,
                asset_filter,
                cfg.utilization_intervals,
                cfg.group_by_value,
                cfg.property_not_available_string,
                cfg.start_date,
                cfg.end_date,
                cfg.interval_group_function)
            filled_utilization = backfill_unused_assets_utilization(
                grouped_utilization=grouped_utilization, 
                unused_asset_identifiers=unused_asset_identifiers,
                date=cfg.utilization_for_date
            )
            for utilization in filled_utilization:
                asset_id = utilization[0]
                date_str = utilization[1]
                timestamp_adjusted_by_timezone = adjust_datetime(
                    date_time_str=date_str, 
                    whp_start_time=policy_start_time
                )
                asset = asset_id_to_asset_map[asset_id]
                
                vendor, model, serial_number = get_vendor_model_serial_number_settings(asset)
                
                path= f"Assets.{vendor}.{model}.{serial_number}.{cfg.tag_middle_segment}.Daily.All.{category_prefix_for_tag}"
                tag_value = TagValue(value=str(filled_utilization[utilization]['percentage']), type=TagType.DOUBLE)
                timestamped_tag_value = TimestampedTagValue(timestamp=timestamp_adjusted_by_timezone, value=tag_value)
                tag_update = TagUpdate(path= path,
                                       updates=[timestamped_tag_value], 
                                       workspace=asset['workspace'])
                tag_updates.append(tag_update)
        chunked_tag_updates = [tag_updates[i:i + tag_chunk_size] for i in range(0, len(tag_updates), tag_chunk_size)]
        for tag_updates in chunked_tag_updates:
            await tags_api.update_tag_current_values(updates=tag_updates)
            await asyncio.sleep(cfg.delay_between_tag_batches)
    await tags_api.api_client.close()

## System and Workspace Utilization

In [ ]:
async def update_system_and_workspace_utilization_tags(utilization=None, controllers_by_workspace_id={}):
    utilization_filter, category_prefix_for_tag = get_utilization_settings(utilization)

    tags_api = TagsApi()
    for ws_id in controllers_by_workspace_id:
        number_of_uncontrolled_systems_in_workspace = 0
        workspace_utilization = []
        utilization_of_systems_in_ws = []
        sys_controllers = controllers_by_workspace_id[ws_id]["controllers_with_alias"]
        virtual_systems = controllers_by_workspace_id[ws_id]["virtual_systems"]

        if not len(sys_controllers):
            workspace_utilization.append(0)
        system_tags_metadata = []
        virtual_system_tags_metadata = []
        # system controllers
        for sys_controller in sys_controllers:
            alias = sys_controller.get("alias", "")
            minion_id = sys_controller['location'].get("minion_id", "")
            properties = {
                "name": alias,
                "asset_identifier": sys_controller["id"],
                "minion_id": minion_id,
                "workspace": str(sys_controller["workspace"]),
                "displayName": f"{alias} - Utilization Daily All"
            }
            tag_path = f"{minion_id}.{cfg.tag_middle_segment}.Daily.All.{category_prefix_for_tag}"
            system_tags_metadata.append({"path": tag_path, "properties": properties})

        # virtual system
        number_of_uncontrolled_systems_in_workspace = len(virtual_systems)
        for virtual_system in virtual_systems:
            minion_id = virtual_system.get("id", "")
            v_system_alias = virtual_system.get("alias", "")
            properties = {
                "name": v_system_alias,
                "asset_identifier": "",
                "minion_id": minion_id,
                "workspace": virtual_system["workspace"],
                "displayName": f"{v_system_alias} - Utilization Daily All"
            }

            tag_path = f"{minion_id}.{cfg.tag_middle_segment}.Daily.All.{category_prefix_for_tag}"
            virtual_system_tags_metadata.append({"path": tag_path, "properties": properties})

        await patch_virtual_system_utilization(tags_metadata=virtual_system_tags_metadata, date_str=cfg.utilization_for_date)
        await ensure_tags_exists(tags_metadata=system_tags_metadata)

        sys_controllers_filters_list = create_asset_filter(sys_controllers)
        sys_controller_id_to_sys_controller_map = {}
        for sys_controller in sys_controllers:
            sys_controller_id_to_sys_controller_map[sys_controller['id']] = sys_controller

        tag_updates = []
        for sys_controller_filter in sys_controllers_filters_list:
            grouped_utilization, unused_asset_identifiers = await compute_utilization(
                utilization_filter,
                sys_controller_filter,
                cfg.utilization_intervals,
                cfg.group_by_value,
                cfg.property_not_available_string,
                cfg.start_date,
                cfg.end_date,
                cfg.interval_group_function)
            filled_utilization = backfill_unused_assets_utilization(
                grouped_utilization,
                unused_asset_identifiers,
                cfg.utilization_for_date
            )
            for utilization in filled_utilization:
                system_id = utilization[0]
                date_str = utilization[1]
                timestamp_adjusted_by_timezone = adjust_datetime(
                    date_time_str=date_str,
                    whp_start_time=policy_start_time
                )
                sys_controller = sys_controller_id_to_sys_controller_map[system_id]
                tag_value = TagValue(value=str(filled_utilization[utilization]['percentage']), type=TagType.DOUBLE)
                timestamped_tag_value = TimestampedTagValue(timestamp=timestamp_adjusted_by_timezone, value=tag_value)
                path = f"{sys_controller['location']['minion_id']}.{cfg.tag_middle_segment}.Daily.All.{category_prefix_for_tag}"

                tag_update = TagUpdate(path=path,
                                       updates=[timestamped_tag_value],
                                       workspace=sys_controller['workspace'])
                tag_updates.append(tag_update)
                workspace_utilization.append(filled_utilization[utilization]['percentage'])
        workspace_utilization.extend([0] * number_of_uncontrolled_systems_in_workspace)
        chunked_tag_updates = [tag_updates[i:i + tag_chunk_size] for i in range(0, len(tag_updates), tag_chunk_size)]
        for tag_updates in chunked_tag_updates:
            await tags_api.update_tag_current_values(updates=tag_updates)
            await asyncio.sleep(cfg.delay_between_tag_batches)

        await ensure_tags_exists([{
            'path': f"Workspace.{ws_id}.{cfg.tag_middle_segment}.Daily.All.{category_prefix_for_tag}",
            'properties': {
                'workspace': ws_id,
                'displayName': f"{translate_ids_to_names([ws_id], api_key)[0]} - Utilization Daily All"
            }}])

        timestamp_adjusted_by_timezone = adjust_datetime(
            date_time_str=cfg.utilization_for_date,
            whp_start_time=policy_start_time
        )
        ws_utilization_precentage = sum(workspace_utilization) / len(workspace_utilization)
        tag_value = TagValue(value=str(ws_utilization_precentage), type=TagType.DOUBLE)
        timestamped_tag_value = TimestampedTagValue(timestamp=timestamp_adjusted_by_timezone, value=tag_value)
        tag_update = TagUpdate(path=f"Workspace.{ws_id}.{cfg.tag_middle_segment}.Daily.All.{category_prefix_for_tag}",
                               updates=[timestamped_tag_value], workspace=ws_id)
        update = [tag_update]
        await tags_api.update_tag_current_values(updates=update)
        await asyncio.sleep(cfg.delay_between_tag_batches)
    await tags_api.api_client.close()

## Get assets

In [ ]:
assets_filter = " OR ".join(f'workspace = "{wid}"' for wid in cfg.get_target_workspace_ids())
query_assets_request = QueryAssetsRequest(
    filter=assets_filter
)
all_assets = await fetch_all_assets(
    query_request=query_assets_request,
)

## Push asset utilization to tags

In [ ]:
await update_asset_utilization_tags(assets=all_assets)
await update_asset_utilization_tags(assets=all_assets, utilization="configuration")

### Get systems

In [ ]:
systems_filter = " OR ".join(f'workspace = "{wid}"' for wid in cfg.get_target_workspace_ids())

query_systems_request = QuerySystemsRequest(
    projection='new(id,alias,workspace)',
    filter=systems_filter
)

all_systems = await fetch_all_systems(
    query_request=query_systems_request,
)

### Get system controllers(assets)

In [ ]:
system_controller_filter_string = create_minion_filter(all_systems, workspaces=cfg.get_target_workspace_ids())

query_system_controller_request = QueryAssetsRequest(skip=0, take=1000, filter=system_controller_filter_string)
all_system_controllers = await fetch_all_assets(query_system_controller_request)

### Prepare workspace id to controllers map

In [ ]:
workspace_system_map = build_workspace_system_map(all_systems, all_system_controllers)

### Push workspace and systems utilization to tags

In [ ]:
await update_system_and_workspace_utilization_tags(controllers_by_workspace_id=workspace_system_map)
await update_system_and_workspace_utilization_tags(utilization='configuration', controllers_by_workspace_id=workspace_system_map)